# SHA-2

## Wypełnianie wiadomości $M$

In [1]:
def padding(x):
    k = len(x)
    if len(x)%512 <= 447:
        x += "1"
        zeros_count = 512 - len(x)%512 - 64
        x += "0"*zeros_count
        zeros_count = 512 - 448 - len(bin(k)[2:])
        x += "0"*zeros_count
        x += bin(k)[2:]
        return x
    elif len(x)%512 > 447:
        x += "1"
        zeros_count = 512 - len(x)%512
        x += "0"*zeros_count
        x += "0"*448
        zeros_count = 512 - 448 - len(bin(k)[2:])
        x += "0"*zeros_count
        x += bin(k)[2:]
        return x

## Funkcje logiczne

In [2]:

def SHR(x, y):
   for i in range(y):
       x = "0" + x[:-1]
   return x

def ROTR(x, y):
   for i in range(y):
       x = x[-1] + x[:-1]
   return x


        
def XOR(a, b):
    return ''.join('0' if a[i] == b[i] else '1' for i in range(len(a)))

def AND(a, b):
    return ''.join('1' if a[i] == b[i] == "1" else '0' for i in range(len(a)))

def NOT(x):
    return ''.join('1' if x[i] == '0' else '0' for i in range(len(x)))



def signum_1(x):
    return XOR( XOR(ROTR(x, 17), ROTR(x, 19)), SHR(x, 10))

def signum_0(x):
    return XOR( XOR(ROTR(x, 7), ROTR(x, 18)), SHR(x, 3))

## Funkcje wewnątrz rundy 
#### (dodawanie w $\mathbb{Z}_{2^{32}}$)

In [3]:
def MAJ(a, b, c):
    return XOR(XOR(AND(a, b), AND(a, c)), AND(b, c))
def Ep_0(a):
    return XOR(XOR(ROTR(a, 2), ROTR(a, 13)), ROTR(a, 22))
def Ep_1(e):
    return XOR(XOR(ROTR(e, 6), ROTR(e, 11)), ROTR(e, 25))
def CH(e, f, g):
    return XOR(AND(e, f), AND(NOT(e), g))

# Adding in Z_2^32

def ADD(a, b):
    a = int(a, 2)
    b = int(b, 2)
    x = bin((a + b)%(2**32))[2:]
    x = x.zfill(32)
    return x

## Wzor na Wartość $W_t$

In [4]:
def split_512(x):
    M = []
    for m in range(len(x)//512):
        M.append(x[m*512:(m+1)*512])
    return M

def divblock(M):
    B = M[0:512]
    W_i = [B[32*i:32*i+32] for i in range(0,16)]
    B = W_i
    return B

def Wt(Wt_2, Wt_7, Wt_15, Wt_16):
    W7 = int(Wt_7, 2)
    W16 = int(Wt_16, 2)
    W_i = bin((int(signum_1(Wt_2), 2) + W7 + int(signum_0(Wt_15), 2) + W16)%(2**32))[2:].zfill(32)
    return W_i

## Wartości $K_i$ i $H_i$

In [5]:
import math

def fractional_to_binary(n):
    fractional_part = n - int(n)

    FRAC_N = []

    while fractional_part != 0 and len(FRAC_N) < 32:
        fractional_part *= 2
        digit = int(fractional_part)
        FRAC_N.append(str(digit))
        fractional_part -= digit

    binary_fractional_part_str = ''.join(FRAC_N)
    
    return binary_fractional_part_str

# List of 64 prime numbers

Prime = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]


# Generating values K and H

H = [i for i in range(8)]
K = [i for i in range(64)]

for i in range(8):
    H[i] = fractional_to_binary(math.sqrt(Prime[i]))

for i in range(64):
    K[i] = fractional_to_binary(math.cbrt(Prime[i]))


# SHA-2

In [6]:
def SHA_2(m = ""):
    
# ASCII standard conversion

    M = ''.join(format(ord(char), '08b') for char in m)
    M = split_512(padding(M))
    
# Block [0-n]

    H_ = H.copy()

    for n in range(len(M)):
        if n == 0:
            V = H_.copy()
            
# V = [a, b, c, d, e, f, g, h]
        
# Make (W_0 - W_63)

        W = divblock(M[n])
        for i in range(16, 64):
            W.append(Wt(W[i-2],W[i-7], W[i-15], W[i-16]))
            
# Round [0-63]

        for i in range(64):
            T1 = ADD(ADD(ADD(ADD(CH(V[4], V[5], V[6]), V[7]), Ep_1(V[4])), W[i]), K[i])
            T2 = ADD(MAJ(V[0], V[1], V[2]), Ep_0(V[0]))
            V[7] = V[6] 
            V[6] = V[5]
            V[5] = V[4]
            V[4] = ADD(V[3], T1)
            V[3] = V[2]
            V[2] = V[1]
            V[1] = V[0]
            V[0] = ADD(T1, T2)

        for i in range(len(V)):
            V[i] = ADD(V[i], H_[i])
            
# Initializing the H value (a-f) for the next block

        H_ = V.copy()

# Make hash from vector H = V

    xhash = ''
    for i in H_:
        xhash += i
    return f"The SHA2 hash of the data is: {hex(int(xhash, 2))[2:].zfill(64)}"

In [7]:
SHA_2('')

'The SHA2 hash of the data is: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'

In [8]:
from hashlib import sha256

input_ = ""
sha256(input_.encode('utf-8')).hexdigest()

'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'